## Messing around with PostgreSQL
### Zetcode approach using Pandas
http://zetcode.com/db/postgresqlpythontutorial/

In [1]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import sys
#import pandas as pd

Start postgresql server.  Launch from launchpad, or

$ postgres -D /Users/rockson/Library/Application\ Support/Postgres/var-9.5

In [2]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'cars_db'
username = 'rockson'

## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://rockson@localhost/cars_db


In [3]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


### Find version

In [4]:
con = None
try:
    # connect:
    con = psycopg2.connect(database = dbname, user = username)
    cur = con.cursor()
    # find version
    cur.execute('SELECT version()')
    ver = cur.fetchone()
    print ver

except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
        
finally:    
    if con:
        con.close()

('PostgreSQL 9.5.4 on x86_64-apple-darwin14.5.0, compiled by Apple LLVM version 7.0.0 (clang-700.1.76), 64-bit',)


### Inserting data

In [5]:
con = None
try:
    # connect:
    con = psycopg2.connect(database = dbname, user = username)
    cur = con.cursor()
    # insert data:
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name VARCHAR(20), Price INT)")
    cur.execute("INSERT INTO Cars VALUES(1,'Audi',52642)")
    cur.execute("INSERT INTO Cars VALUES(2,'Mercedes',57127)")
    cur.execute("INSERT INTO Cars VALUES(3,'Skoda',9000)")
    cur.execute("INSERT INTO Cars VALUES(4,'Volvo',29000)")
    cur.execute("INSERT INTO Cars VALUES(5,'Bentley',350000)")
    cur.execute("INSERT INTO Cars VALUES(6,'Citroen',21000)")
    cur.execute("INSERT INTO Cars VALUES(7,'Hummer',41400)")
    cur.execute("INSERT INTO Cars VALUES(8,'Volkswagen',21600)")
    con.commit()
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
finally:
    if con:
        con.close()

Open psql terminal, run some terminal commands

connect to db

$ \c birth_db2

NOTE: SQL commands are capitalized.  They need to be capitalized, and terminated by a semi-colon.  Can write a multi-line command, prompt will show a '-' instead of a '='.

Check list of databases

$ \l

Check list of tables in current database

$ \dt
$ \d+ (for full info?)

Column names /info in a given table

$ \d+ <table name>

Run a SQL query

$ SELECT * FROM Cars;

Connect to a different database as user

$ \c birth_db rockson

Delete a database

$ DROP DATABASE birth_db;

Delete table

$ DROP TABLE Cars;

Delete data from a table

$ DELETE FROM Cars;

Disconnect from database 

$ \q

Delete database or table

$ DROP DATABASE [ IF EXISTS ] name;
$ DROP TABLE [ IF EXISTS ] name;

### Write same table using executemany() command

In [6]:
autos = (
    (1, 'Audix', 52642),
    (2, 'Mercedesx', 57127),
    (3, 'Skodax', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Citroen', 21000),
    (7, 'Hummer', 41400),
    (8, 'Volkswagen', 21600)
)

con = None
try: 
    # connect
    con = psycopg2.connect(database = dbname, user = username)
    cur = con.cursor()
    # Insert data
    cur.execute("DROP TABLE IF EXISTS Autos")
    cur.execute("CREATE TABLE Autos(Id INT PRIMARY KEY, Name TEXT, Price INT)")
    query = "INSERT INTO Autos (Id, Name, Price) VALUES (%s, %s, %s)"
    cur.executemany(query, autos)  
    con.commit()
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
finally:
    if con:
        con.close()

### Retreive data from table

In [15]:
con = None
try:
    # connect
    con = psycopg2.connect(database = dbname, user = username)
    cur = con.cursor()
    # retreive data
    cur.execute("SELECT * FROM Autos")
    rows = cur.fetchall() # return all data from SQL query as a tuple of tuples
    # row = cur.fetchone() # return one at a time
    # print data  
    for row in rows:
        print row
except psycopg2.DatabaseError, e:
    print 'Error %s' %e
    sys.exit(1)
finally:
    if con:
        con.close()

(1, 'Audix', 52642)
(2, 'Mercedesx', 57127)
(3, 'Skodax', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
(6, 'Citroen', 21000)
(7, 'Hummer', 41400)
(8, 'Volkswagen', 21600)


### Dictionary cursor

In [33]:
import psycopg2.extras # not sure if I need this imprt

con = None
try:
    con = psycopg2.connect(database = dbname, user = username) 
    
    cursor = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    cursor.execute("SELECT * FROM Cars")
    
    rows = cursor.fetchall()
    for row in rows:
        print "%s %s %s" % (row["id"], row["name"], row["price"])
        # note that capitalization isn't important
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
  
finally:
    
    if con:
        con.close()

2 Mercedes 57127
3 Skoda 9000
4 Volvo 29000
5 Bentley 350000
6 Citroen 21000
1 Audi 62300
7 Hummer 62301
8 Volkswagen 62301


### Parametrized queries
Note that updated rows shift to bottom of table

In [32]:
con = None

uId = 6
uPrice = 62301

try:     
    con = psycopg2.connect(database = dbname, user = username) 
    cur = con.cursor()
    
    #cur.execute("UPDATE Cars SET Price=%s WHERE Id>%s", (uPrice, uId))        
    cur.execute("UPDATE Cars SET Price=%(price)s WHERE Id>%(id)s", {'price':uPrice, 'id':uId})        
    con.commit()
    print "Number of rows updated: %d" % cur.rowcount
    
except psycopg2.DatabaseError, e:
    if con:
        con.rollback()
    print 'Error %s' % e    
    sys.exit(1)
finally:
    if con:
        con.close()

Number of rows updated: 2


### Inserting images into a database -- SKIP

### Reading images from a database -- SKIP

### Metadata

In [43]:
con = None
try:
    con = psycopg2.connect(database = dbname, user = username) 
    cur = con.cursor()
    cur.execute('SELECT * FROM Cars')
    col_names = [cn[0] for cn in cur.description]
    rows = cur.fetchall()
    
    print "%s %-10s %s" % (col_names[0], col_names[1], col_names[2])
    for row in rows:    
        print "%2s %-10s %s" % row
        
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)
finally:
    if con:
        con.close()

id name       price
 2 Mercedes   57127
 3 Skoda      9000
 4 Volvo      29000
 5 Bentley    350000
 6 Citroen    21000
 1 Audi       62300
 7 Hummer     62301
 8 Volkswagen 62301


### Export data

In [37]:
con = None
fout = None
try:     
    con = psycopg2.connect(database=dbname, user=username)     
    cur = con.cursor()
    fout = open('cars.sql', 'w')
    cur.copy_to(fout, 'cars', sep="|")                        
    
except psycopg2.DatabaseError, e:
    print 'Error %s' % e    
    sys.exit(1)

except IOError, e:    
    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    if con:
        con.close()
    if fout:
        fout.close() 

### Import data
For this particular example, need to clear data from table first

$ DELETE FROM cars;

In [42]:
con = None
f = None

try:
    con = psycopg2.connect(database=dbname, user=username) 
    cur = con.cursor()
    f = open('cars.sql', 'r')
    cur.copy_from(f, 'cars', sep="|")                    
    con.commit()
    
except psycopg2.DatabaseError, e:    
    if con:
        con.rollback()
    print 'Error %s' % e    
    sys.exit(1)
except IOError, e:    
    if con:
        con.rollback()
    print 'Error %s' % e   
    sys.exit(1)
    
finally:
    if con:
        con.close()
    if f:
        f.close()  

### Transactions

IN the psycopg2 module, a transaction is started by the first command to a connection cursor.  Subsequent commands fall under this same transaction.    

To terminate at transaction, it is necessary to run con.commit() or con.rollback().

Instead of an explicit commit(), can specify autocommit, 

con.autocommit = True